In [45]:
import pandas as pd
from tqdm import tqdm
import random
from  datetime import datetime
import phonenumbers
from clean_phone import clean_phone
from clean_phone import _check_phone
from clean_phone import validate_phone

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


def extract_columns_from_ddl(table_ddl):
    column_names = [line.split()[0] for line in table_ddl.strip().split('\n')]
    return  column_names

def generate_dummy_data_df(n):
    data = []
    for _ in range(n):
        data.append({
            "PARTY_PHONE_ID": random.randint(1, 1000),
            "PARTY_ID": random.randint(1, 1000),
            "COUNTRY_ID": f'Country_{random.randint(1, 100)}',
            "PARTY_PHONE_TYPE_ID": random.randint(1, 10),
            "PHONE_NUMBER": f'{random.randint(100000, 999999)}',
            "PHONE_NUMBER_PREFIX": f'+{random.randint(10, 99)}',
            "SRC_ID": f'SRC_{random.randint(1, 100)}',
            "SRC_SYS_ID": f'SYS_{random.randint(1, 100)}',
            "DEL_FLAG": random.randint(0, 1),
            "INSERT_DATETIME": datetime.now().date(),
            "INS_PROCESS_ID": f'Process_{random.randint(1, 100)}',
            "UPDATE_DATETIME":  pd.to_datetime('2024-01-05'),
            "UPD_PROCESS_ID": f'Process_{random.randint(101, 200)}',
            "UPD_EFF_DATE": datetime.now().date()
        })
    return pd.DataFrame(data)

def MTCH_PT_PHONE_definition(table_ddl):

    columns = extract_columns_from_ddl(table_ddl)
    return pd.DataFrame(columns=columns)

def insert_phones(df):
    transformed_df = pd.DataFrame(
        {
        'PARTY_PHONE_ID': df['PARTY_PHONE_ID'],
        'PARTY_ID': df['PARTY_ID'],
        'COUNTRY_ID': df['COUNTRY_ID'],
        'PARTY_PHONE_TYPE_ID': df['PARTY_PHONE_TYPE_ID'],
        'PHONE_NUMBER': df['PHONE_NUMBER'],
        'PHONE_NUMBER_PREFIX': df['PHONE_NUMBER_PREFIX'],
        'SRC_ID': df['SRC_ID'],
        'SRC_SYS_ID': df['SRC_SYS_ID'],
        'DELETE_FLAG': df['DEL_FLAG'],
        'INSERT_DATETIME': df['INSERT_DATETIME'],
        'INSERT_PROCESS_ID': df['INS_PROCESS_ID'],
        'UPDATE_DATETIME': df['UPDATE_DATETIME'],
        'UPDATE_DATETIME_PROCESS_ID': df['UPD_PROCESS_ID'],
        'UPDATE_DATETIME_EFFECTIVE_DATE': df['UPD_EFF_DATE']
        }
        
    )
    return transformed_df

def validate_phone_number(number):
    try:
        parsed_number = phonenumbers.parse(str(number), None)
        return phonenumbers.is_valid_number(parsed_number)
    except phonenumbers.NumberParseException:
        return False

def unified_phone_id(df,column_for_unification,group_by_column, phone_key, result_column):
    """
        df :  pd.DataFrame() with party phone numbers
        column_for_unification :  column which contains valid phone numbers for unification
        group_by_column :  key, based on which the data will be grouped 
        phone_key : primary key, which identifies the phone
        result_column : name of the result column, where the result values will be added
    """

    filtered_df = df[df[column_for_unification].notnull() & ~df[column_for_unification].isna()]
    grouped_phones = filtered_df.groupby(group_by_column)[phone_key].apply(list)
    df[result_column] = df[group_by_column].map(grouped_phones)
    
    return df

def impute_null_phone_numbers(df, column, value):
    for index, row in tqdm(df[df[column].isnull()].iterrows(),desc = "Null value impute is running ..."):
        df.loc[index,column] = value
    return df 


def list_to_string_transformation(df: pd.DataFrame, column: str):
    for index, row in tqdm(df.iterrows(), desc="List transformation is running ..."):
        # Check if the entry is iterable (e.g., a list), if not, skip the iteration.
        if not isinstance(row[column], list):
            #print(f"Non-iterable item at index {index}: {row['UNIFICATION_PARTY_PHONE_ID']}")
            continue
        
        resulted_string = ""
        for val in row[column]:
            # First method
            values_as_string = ["'{}'".format(int(val)) for val in row[column]]
            resulted_string = ', '.join(values_as_string)
            df.loc[index, column]
            
            # Properly concatenate the string with each value.
        df.loc[index, column] = resulted_string
    return df



## Table creation

In [16]:
ddl = """
    SORURCE_PHONE_PREFIX VARCHAR(255),
    SORURCE_PHONE_NUMBER VARCHAR(255),
    MATCHING_PHONE_NUMBER VARCHAR(255),
    MASTER_PARTY_PHONE_ID INTEGER,
    UNIFICATION_PARTY_PHONE_ID INTEGER,
    SOURCE_IDENTIFIER VARCHAR(255),
    SOURCE_SYSTEM_IDENTIFIER VARCHAR(255),
    DELETE_FLAG INTEGER,
    INSERT_DATETIME DATE,
    INSERT_PROCESS_ID VARCHAR(255),
    UPDATE_DATETIME DATE,
    UPDATE_DATETIME_PROCESS_ID VARCHAR(255),
    UPDATE_DATETIME_EFFECTIVE_DATE DATE
"""
MTCH_PT_PHONE = MTCH_PT_PHONE_definition(ddl)
PARTY_PHONE = generate_dummy_data_df(32029)
PARTY_PHONE = PARTY_PHONE.drop_duplicates()

#Type correction
MTCH_PT_PHONE['UPDATE_DATETIME'] = pd.to_datetime(MTCH_PT_PHONE['UPDATE_DATETIME'], errors='coerce')
MTCH_PT_PHONE['INSERT_DATETIME'] = pd.to_datetime(MTCH_PT_PHONE['INSERT_DATETIME'], errors='coerce')

print(MTCH_PT_PHONE.shape)
print(PARTY_PHONE.shape)

(0, 13)
(32029, 14)


## MTCH table population
 - Insert not validated data 

In [17]:

print(MTCH_PT_PHONE.shape)
last_checked_date = pd.to_datetime('2024-01-10')
filtered_MTCH_PT = PARTY_PHONE[PARTY_PHONE['UPDATE_DATETIME']<last_checked_date]
MTCH_PT_PHONE = pd.concat([MTCH_PT_PHONE, insert_phones(filtered_MTCH_PT)], ignore_index=True)
for index, row in tqdm(MTCH_PT_PHONE.iterrows(),desc = "Phone union"):
     MTCH_PT_PHONE.loc[index,'MATCHING_PHONE_NUMBER_TMP'] = row['PHONE_NUMBER_PREFIX'] + row['PHONE_NUMBER']
print(MTCH_PT_PHONE.shape)

(0, 13)


Phone union: 32029it [00:02, 12145.56it/s]

(32029, 22)


## 1 - First method using is_valid_number()

In [8]:
# Get valid phones
print("MATCHING_PHONE_NUMBER null value count ",MTCH_PT_PHONE.MATCHING_PHONE_NUMBER.isna().sum(),"\n")

for index, row in tqdm(MTCH_PT_PHONE.iterrows(), "phonenumbers method is running ..."):
    # Assuming 'PHONE_NUMBER' is the actual phone number column
    phone_number_str = row['PHONE_NUMBER_PREFIX'] + row['PHONE_NUMBER']
    try:
        phone_number = phonenumbers.parse(phone_number_str, None)
        if phonenumbers.is_valid_number(phone_number):       
            MTCH_PT_PHONE.loc[index, 'MATCHING_PHONE_NUMBER'] = phonenumbers.format_number(phone_number, phonenumbers.PhoneNumberFormat.INTERNATIONAL)   
        else:
            MTCH_PT_PHONE.loc[index, 'MATCHING_PHONE_NUMBER'] = None
    except phonenumbers.NumberParseException:
        MTCH_PT_PHONE.loc[index, 'MATCHING_PHONE_NUMBER'] = None
print("\n")
print("MATCHING_PHONE_NUMBER null value count ",MTCH_PT_PHONE.MATCHING_PHONE_NUMBER.isna().sum())
print("\n")
print("UNIFICATION_PARTY_PHONE_ID null value count ",MTCH_PT_PHONE['UNIFICATION_PARTY_PHONE_ID'].isna().sum())
print("\n")
MTCH_PT_PHONE = unified_phone_id(MTCH_PT_PHONE,'MATCHING_PHONE_NUMBER','PARTY_ID', 'PARTY_PHONE_ID', 'UNIFICATION_PARTY_PHONE_ID')
print("UNIFICATION_PARTY_PHONE_ID null value count ",MTCH_PT_PHONE['UNIFICATION_PARTY_PHONE_ID'].isna().sum())
print("\n")

MTCH_PT_PHONE = impute_null_phone_numbers(MTCH_PT_PHONE,'MATCHING_PHONE_NUMBER',None)
print("MATCHING_PHONE_NUMBER null value count ",MTCH_PT_PHONE.MATCHING_PHONE_NUMBER.isna().sum())


# filtered_df = MTCH_PT_PHONE[MTCH_PT_PHONE['MATCHING_PHONE_NUMBER'].notnull()] # take only valid number
# grouped_phones = filtered_df.groupby('PARTY_ID')['PARTY_PHONE_ID'].apply(list)
# MTCH_PT_PHONE['UNIFICATION_PARTY_PHONE_ID'] = MTCH_PT_PHONE['PARTY_ID'].map(grouped_phones)

    



#MTCH_PT_PHONE[~MTCH_PT_PHONE.MATCHING_PHONE_NUMBER.isna()][['MATCHING_PHONE_NUMBER','PHONE_NUMBER_PREFIX', 'PHONE_NUMBER']]


MATCHING_PHONE_NUMBER null value count  32029 



phonenumbers method is running ...: 32029it [00:08, 3868.17it/s]




MATCHING_PHONE_NUMBER null value count  31311


UNIFICATION_PARTY_PHONE_ID null value count  32029


UNIFICATION_PARTY_PHONE_ID null value count  15554




Null value impute is running ...: 31311it [00:06, 4818.64it/s]

MATCHING_PHONE_NUMBER null value count  31311


## Second option 

In [18]:

print("MATCHING_PHONE_NUMBER null value count ",MTCH_PT_PHONE.MATCHING_PHONE_NUMBER.isna().sum(),"\n")

#res = []
for index, row in tqdm(MTCH_PT_PHONE.iterrows(), "validate_phone method is running ..."):
    #res.append(_check_phone(row['MATCHING_PHONE_NUMBER_TMP'],False))
    phone_number_str = row['PHONE_NUMBER_PREFIX'] + row['PHONE_NUMBER']
    if validate_phone(phone_number_str):
        MTCH_PT_PHONE.loc[index, 'MATCHING_PHONE_NUMBER'] = phonenumbers.format_number(phonenumbers.parse(phone_number_str, None), phonenumbers.PhoneNumberFormat.INTERNATIONAL)   
    else:
        MTCH_PT_PHONE.loc[index, 'MATCHING_PHONE_NUMBER'] = None 

print("MATCHING_PHONE_NUMBER null value count ",MTCH_PT_PHONE.MATCHING_PHONE_NUMBER.isna().sum(),"\n")

print("UNIFICATION_PARTY_PHONE_ID null value count ",MTCH_PT_PHONE['UNIFICATION_PARTY_PHONE_ID'].isna().sum())

MTCH_PT_PHONE = unified_phone_id(df = MTCH_PT_PHONE,
                                column_for_unification = 'MATCHING_PHONE_NUMBER',
                                group_by_column = 'PARTY_ID',
                                phone_key = 'PARTY_PHONE_ID',
                                result_column =  'UNIFICATION_PARTY_PHONE_ID')

print("UNIFICATION_PARTY_PHONE_ID null value count ",MTCH_PT_PHONE['UNIFICATION_PARTY_PHONE_ID'].isna().sum())

MTCH_PT_PHONE = impute_null_phone_numbers(MTCH_PT_PHONE,'MATCHING_PHONE_NUMBER',None)
print("MATCHING_PHONE_NUMBER null value count ",MTCH_PT_PHONE.MATCHING_PHONE_NUMBER.isna().sum(),"\n")



MATCHING_PHONE_NUMBER null value count  32029 



validate_phone method is running ...: 32029it [00:07, 4358.03it/s]


MATCHING_PHONE_NUMBER null value count  28411 

UNIFICATION_PARTY_PHONE_ID null value count  32029
UNIFICATION_PARTY_PHONE_ID null value count  729


Null value impute is running ...: 28411it [00:05, 4941.85it/s]

MATCHING_PHONE_NUMBER null value count  28411 



## List transform to store data in KBC

In [ ]:
MTCH_PT_PHONE = list_to_string_transformation(MTCH_PT_PHONE,'UNIFICATION_PARTY_PHONE_ID' )

## Add column to matching party table 

In [60]:
MTCH_PT = pd.read_csv('mtch_pt.csv')
MTCH_PT = MTCH_PT.rename(columns={"MATCHING_MFO": "UNIFICATION_PARTY_PHONE_ID"})
#merged_df = pd.merge(MTCH_PT, MTCH_PT_PHONE[['PARTY_ID', 'UNIFICATION_PARTY_PHONE_ID']], on='PARTY_ID', how='left')

merged_df = pd.merge(MTCH_PT, 
                     MTCH_PT_PHONE[['PARTY_ID', 'UNIFICATION_PARTY_PHONE_ID']], 
                     on='PARTY_ID', 
                     how='left',
                     suffixes=('', '_from_phone'))

# Update UNIFICATION_PARTY_PHONE_ID in MTCH_PT with the values from MTCH_PT_PHONE
MTCH_PT['UNIFICATION_PARTY_PHONE_ID'] = merged_df['UNIFICATION_PARTY_PHONE_ID_from_phone']

### To csv and load in the next stage

In [ ]:
MTCH_PT.to_csv('MTCH_PT_phones.csv')

In [ ]:
id_list = list(MTCH_PT['PARTY_ID'].head(20))
MTCH_PT_PHONE[MTCH_PT_PHONE['PARTY_ID'].isin(id_list)]

In [ ]:
id_list = list(MTCH_PT['PARTY_ID'].head(20))
print(MTCH_PT_PHONE)

In [ ]:
MTCH_PT['UNIFICATION_PARTY_PHONE_ID'].head(100)
MTCH_PT[['PARTY_ID', 'UNIFICATION_PARTY_PHONE_ID']]

In [ ]:
merged_df.head(1000)
MTCH_PT['UNIFICATION_PARTY_PHONE_ID'] = merged_df['UNIFICATION_PARTY_PHONE_ID']
MTCH_PT.shape

## Function testing 

In [ ]:
df_tmp = MTCH_PT_PHONE.copy()
print(df_tmp['UNIFICATION_PARTY_PHONE_ID'].head(10))
df_tmp = list_to_string_transformation(df_tmp,'UNIFICATION_PARTY_PHONE_ID' )

print(df_tmp['UNIFICATION_PARTY_PHONE_ID'].head(100))




for index, row in tqdm(df_tmp.iterrows(), desc="List transform is running ..."):
    # Check if the entry is iterable (e.g., a list), if not, skip the iteration.
    if not isinstance(row['UNIFICATION_PARTY_PHONE_ID'], list):
        #print(f"Non-iterable item at index {index}: {row['UNIFICATION_PARTY_PHONE_ID']}")
        continue
    
    resulted_string = ""
    for val in row['UNIFICATION_PARTY_PHONE_ID']:
        # First method
        values_as_string = ["'{}'".format(int(val)) for val in row['UNIFICATION_PARTY_PHONE_ID']]
        resulted_string = ', '.join(values_as_string)
        df_tmp.loc[index, 'UNIFICATION_PARTY_PHONE_ID']
        
        # Properly concatenate the string with each value.
    df_tmp.loc[index, 'UNIFICATION_PARTY_PHONE_ID'] = resulted_string
    

### Mapping testing 

In [ ]:

# x = MTCH_PT_PHONE['PARTY_ID'].value_counts()
# type(x)
# df = pd.DataFrame({
#     "id": list(set(MTCH_PT_PHONE['PARTY_ID'])),
#     "count": x
# })
# df = df[df["count"]>1]
# df.shape

# print(df[df["id"]==399])
# print(MTCH_PT_PHONE[MTCH_PT_PHONE.PARTY_ID == 399])

# df_copy = MTCH_PT_PHONE.copy()
# print(df_copy.shape)
# df_copy = df_copy[df_copy["PARTY_ID"].isin(df['id'])]
# print(df_copy.shape)
# grouped_phones_tmp = df_copy.groupby('PARTY_ID')['PARTY_PHONE_ID'].apply(list)
# df_copy['UNIFICATION_PARTY_PHONE_ID'] = df_copy['PARTY_ID'].map(grouped_phones_tmp)

# print(df_copy['UNIFICATION_PARTY_PHONE_ID'].head(200))

# grouped_phones_tmp = MTCH_PT_PHONE[MTCH_PT_PHONE["PARTY_ID"] in df['id']].groupby('PARTY_ID')['PARTY_PHONE_ID'].apply(list)
# MTCH_PT_PHONE['UNIFICATION_PARTY_PHONE_ID'] = MTCH_PT_PHONE['PARTY_ID'].map(grouped_phones)
# print(MTCH_PT_PHONE['UNIFICATION_PARTY_PHONE_ID'].head(200))

False    898
True     102
Name: isValid, dtype: int64


In [ ]:
MTCH_PT_PHONE[MTCH_PT_PHONE['isValid'] == False][['isValid','MATCHING_PHONE_NUMBER_TMP', 'PHONE_NUMBER_PREFIX', 'PHONE_NUMBER']].head(100)

In [31]:
ind_list_isValid = []
ind_list_isMatched = []
for index, row in MTCH_PT_PHONE[MTCH_PT_PHONE['isValid'] == True].iterrows():
    ind_list_isValid.append(index)
print("isValid ",len(ind_list_isValid))
for index, row in MTCH_PT_PHONE[~MTCH_PT_PHONE['MATCHING_PHONE_NUMBER'].isna()].iterrows():
    ind_list_isMatched.append(index)
print("isMatched ",len(ind_list_isMatched))
print("\n First group \n")
l = 0
for v in ind_list_isMatched:
    if v not in ind_list_isValid:
        #print("Index ", v, " | ", "Number " , MTCH_PT_PHONE.loc[v, 'MATCHING_PHONE_NUMBER'], " | ","isValid ",MTCH_PT_PHONE.loc[v, 'isValid']," | ", "Check result ", validate_phone_number(row['PHONE_NUMBER_PREFIX'] + row['PHONE_NUMBER']))
        l += 1
print(l)
print("\n Second group \n")
l = 0
for v in ind_list_isValid:
    
    if v not in ind_list_isMatched:
        #print("Index ", v, " | ", "Number " , MTCH_PT_PHONE.loc[v, 'MATCHING_PHONE_NUMBER'], " | ","isValid ",MTCH_PT_PHONE.loc[v, 'isValid']," | ", "Check result ", validate_phone_number(row['PHONE_NUMBER_PREFIX'] + row['PHONE_NUMBER']))
        l += 1
print(l)


isValid  102
isMatched  30

 First group 

30

 Second group 

102


## Testing

In [24]:
import pandas as pd
import numpy as np

validation_list = [
        "555-234-5678", "(555) 234-5678", "555.234.5678", "555/234/5678",
        15551234567, "(1) 555-234-5678", "+1 (234) 567-8901 x. 1234",
        "2345678901 extension 1234", "2345678", "800-299-JUNK", "1-866-4ZIPCAR",
        "123 ABC COMPANY", "+66 91 889 8948", "hello", np.nan, "NULL"
    ]
phone_numbers = pd.DataFrame({
    "phone": validation_list
})
phone_numbers
res = []
for n in phone_numbers:
    try:
        parsed_number = phonenumbers.parse(n, None)
        res.append(phonenumbers.is_valid_number(parsed_number))
    
    except:
         res.append(None)
print(len(validation_list))
print(len(res))
# res_df = pd.DataFrame({"phone":validation_list, "isValid":res})
# res_df

16
1


In [25]:

for n in phone_numbers:
    try:
        parsed_number = phonenumbers.parse(n, None)  # Assuming no default region
        print(phonenumbers.is_valid_number(parsed_number))
    except phonenumbers.NumberParseException as e:
        print(f"Error parsing number {n}: {e}")

Error parsing number phone: (1) The string supplied did not seem to be a phone number.


In [26]:
# Your phone numbers DataFrame
validation_list = [
    "555-234-5678", "(555) 234-5678", "555.234.5678", "555/234/5678",
    15551234567, "(1) 555-234-5678", "+1 (234) 567-8901 x. 1234",
    "2345678901 extension 1234", "2345678", "800-299-JUNK", "1-866-4ZIPCAR",
    "123 ABC COMPANY", "+66 91 889 8948", "hello", np.nan, "NULL"
]
phone_numbers = pd.DataFrame({
    "phone": validation_list
})

validity_results = {}

for n in phone_numbers['phone']:
    try:
        # Convert number to string in case it's not already
        n_str = str(n)
        parsed_number = phonenumbers.parse(n_str, None)  # Replace None with a region code if you have one
        is_valid = phonenumbers.is_valid_number(parsed_number)
        validity_results[n_str] = is_valid
    except phonenumbers.NumberParseException:
        # This means the number couldn't be parsed and is invalid
        validity_results[n_str] = False


print(validity_results)

{'555-234-5678': False, '(555) 234-5678': False, '555.234.5678': False, '555/234/5678': False, '15551234567': False, '(1) 555-234-5678': False, '+1 (234) 567-8901 x. 1234': True, '2345678901 extension 1234': False, '2345678': False, '800-299-JUNK': False, '1-866-4ZIPCAR': False, '123 ABC COMPANY': False, '+66 91 889 8948': True, 'hello': False, 'nan': False, 'NULL': False}


In [27]:
# Function to validate phone number
def validate_phone_number(number):
    try:
        parsed_number = phonenumbers.parse(str(number), None)
        return phonenumbers.is_valid_number(parsed_number)
    except phonenumbers.NumberParseException:
        return False

# Applying validation to each phone number
phone_numbers_df  = phone_numbers.copy()
phone_numbers_df['is_valid'] = phone_numbers_df['phone'].apply(validate_phone_number)

print(phone_numbers_df)

                        phone  is_valid
0                555-234-5678     False
1              (555) 234-5678     False
2                555.234.5678     False
3                555/234/5678     False
4                 15551234567     False
5            (1) 555-234-5678     False
6   +1 (234) 567-8901 x. 1234      True
7   2345678901 extension 1234     False
8                     2345678     False
9                800-299-JUNK     False
10              1-866-4ZIPCAR     False
11            123 ABC COMPANY     False
12            +66 91 889 8948      True
13                      hello     False
14                        NaN     False
15                       NULL     False
